<a href="https://colab.research.google.com/github/Dylan-Lai/distillbert/blob/main/distilBERT_code_CSE_354_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
!cd
!ls

drive  sample_data


In [ ]:
%cd "drive/MyDrive/NLP_Project/data"

/content/drive/.shortcut-targets-by-id/1zuVq9QaS2wxrFrERGmCcxhxs9ubkslrV/NLP_Project/data


In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00


In [ ]:
pip install --upgrade scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
pip install transformers

In [ ]:
import os
import random
import openai
import tiktoken
import json
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
train_data = []
with open("train.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        train_data.append(data)

print(train_data[0])

{'question': 'do iran and afghanistan speak the same language', 'title': 'Persian language', 'answer': True, 'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.'}


In [ ]:
test_data = []
with open("TC354.json", "r", encoding='utf-8-sig') as f:
    for line in f:
        data = json.loads(line)
        test_data.append(data)

print(test_data[10])

{'question': 'Did Perseus kill Medusa?', 'passage': "In the cave Perseus came upon the sleeping Medusa. By viewing Medusa's reflection in his polished shield, he safely approached and cut off her head. From her neck sprang Pegasus ('he who sprang') and Chrysaor ('sword of gold'), the result of Poseidon and Medusa's mating. The other two Gorgons pursued Perseus, but, wearing his helm of darkness, he escaped.From here he proceeded to visit King Atlas who had refused him hospitality; in revenge Perseus turned him to stone (by showing Atlas the severed head).", 'answer': True, 'title': 'Death of a Gorgon'}


Dhruv Verma. (2023). DistilBERT model skeleton code [Source code in Python]. Retrieved from https://colab.research.google.com/drive/1TX0kntDFDemDGapyK7PN7Sj8sXx6fZXR?authuser=1

In [ ]:
class DistillBERT():

  def __init__(self, model='distilbert-base-uncased', num_classes=2):
    # TODO(students): start
    self.model = AutoModelForSequenceClassification.from_pretrained(model)
    self.tokenizer = AutoTokenizer.from_pretrained(model, num_labels = num_classes)
    # TODO(students): end

  def get_tokenizer_and_model(self):
    return self.model, self.tokenizer

In [ ]:
class DatasetLoader():

  def __init__(self, train_data, tokenizer):
    self.data = data
    self.tokenizer = tokenizer

  def tokenize_data(self):  ##train_data is a dict-- access the keys(question, passage, answer, and title) using train_data[i]["name"]
    tokenizedQstnsAndPssgs = []
    labels = []  ##train the passage and the question together in the same encoding -- truncate the answer and title portions
    answers = []
    qstns_pssgs = []

    for line in train_data:
      qstns_pssgs.append("? ".join([line["question"], line["passage"]]))
      answers.append(line["answer"])

    ##separate the question/passage from the answer from the same line, then loop over that to match them up

    for (qstn_pssg, answer) in tqdm(zip(qstns_pssgs, answers), total=len(answers)):
      # TODO(students): start
      encoded_tokens = self.tokenizer.encode_plus(qstn_pssg, max_length = 512, truncation=True, return_tensors = "pt").input_ids
      encoded_tokens = torch.squeeze(encoded_tokens)
      tokenizedQstnsAndPssgs.append(encoded_tokens)
      if(answer == True):
        labels.append(1)
      else:
        labels.append(0)

    labels = torch.tensor(labels)
    tokenizedQstnsAndPssgs = pad_sequence(tokenizedQstnsAndPssgs, batch_first=True)
    dataset = TensorDataset(tokenizedQstnsAndPssgs, labels)
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    processed_dataset = self.tokenize_data()

    data_loader = DataLoader(
        processed_dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )

    return data_loader

In [ ]:
class Trainer():

  def __init__(self, options):
    self.device = options['device']
    self.train_data = options['train_data']
    self.val_data = options['val_data']
    self.batch_size = options['batch_size']
    self.epochs = options['epochs']
    self.save_path = options['save_path']
    self.training_type = options['training_type']
    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0)
    recall = recall_score(labels_flat, pred_flat, zero_division=0)
    f1 = f1_score(labels_flat, pred_flat, zero_division=0)
    accuracy = accuracy_score(labels_flat, pred_flat)
    return precision, recall, f1, accuracy

  def set_training_parameters(self):
    # TODO(students): start

    if self.training_type == 'frozen_embeddings':
      self.model.distilbert.embeddings.word_embeddings.weight.requires_grad = False

    elif self.training_type == 'top_2_training':
      for name, other in self.model.named_parameters():
        if any(char in name for char in ['0', '1', '2', '3']):
          other.requires_grad = False

    elif self.training_type == 'top_4_training':
      for name, other in self.model.named_parameters():
        if any(char in name for char in ['0', '1']):
          other.requires_grad = False

    # TODO(students): end

  def train(self, data_loader, optimizer):
    self.model.train()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0
    total_accuracy = 0

    for batch_idx, (qstns_pssgs, labels) in enumerate(tqdm(data_loader)):
      self.model.zero_grad()
      # TODO(students): start
      qstns_pssgs = qstns_pssgs.to(self.device)
      labels = labels.to(self.device)
      outputs = self.model(qstns_pssgs, labels = labels)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      precision, recall, f1, accuracy = self.get_performance_metrics(outputs.logits.cpu().detach().numpy(), labels.cpu().detach().numpy())
      total_precision += precision
      total_recall += recall
      total_f1 += f1
      total_loss += loss
      total_accuracy += accuracy


      # TODO(students): end

    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)
    accuracy = total_accuracy/len(data_loader)


    return precision, recall, f1, loss, accuracy

  def eval(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0
    total_accuracy = 0

    with torch.no_grad():
      for (qstns_pssgs, labels) in tqdm(data_loader):
        # TODO(students): start

        qstns_pssgs = qstns_pssgs.to(self.device)
        labels = labels.to(self.device)
        outputs = self.model(qstns_pssgs, labels = labels)
        loss = outputs.loss

        precision, recall, f1, accuracy = self.get_performance_metrics(outputs.logits.cpu().detach().numpy(), labels.cpu().detach().numpy())
        total_precision += precision
        total_recall += recall
        total_f1 += f1
        total_loss += loss
        total_accuracy += accuracy


        # TODO(students): end

    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)
    accuracy = total_accuracy/len(data_loader)


    return precision, recall, f1, loss, accuracy

  def save_transformer(self):
    self.model.save_pretrained(self.save_path)
    self.tokenizer.save_pretrained(self.save_path)

  def execute(self):
    last_best = 0
    train_dataset = DatasetLoader(self.train_data, self.tokenizer)
    train_data_loader = train_dataset.get_data_loaders(self.batch_size)
    val_dataset = DatasetLoader(self.val_data, self.tokenizer)
    val_data_loader = val_dataset.get_data_loaders(self.batch_size)
    optimizer = torch.optim.AdamW(self.model.parameters(), lr = 3e-5, eps = 1e-8)
    self.set_training_parameters()
    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss, train_accuracy = self.train(train_data_loader, optimizer)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f} train_accuracy: {train_accuracy:.4f}')
      val_precision, val_recall, val_f1, val_loss, val_accuracy = self.eval(val_data_loader)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f} val_accuracy: {val_accuracy:.4f}')

      if val_f1 > last_best:
        print("Saving model..")
        self.save_transformer()
        last_best = val_f1
        print("Model saved.")



In [ ]:
options = {}
options['batch_size'] = 16
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['train_data'] = train_data
options['val_data'] = test_data
options['save_path'] = "models/distilBERT/" + 'top_4_training'
options['epochs'] = 6
options['training_type'] = 'top_4_training'
trainer = Trainer(options)
trainer.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

Epoch 5: val_loss: 0.1012 val_precision: 0.9758 val_recall: 0.9734 val_f1: 0.9733 val_accuracy: 0.9679
Saving model..
Model saved.


100%|██████████| 590/590 [06:55<00:00,  1.42it/s]


Epoch 6: train_loss: 0.1428 train_precision: 0.9547 train_recall: 0.9585 train_f1: 0.9542 train_accuracy: 0.9457


100%|██████████| 590/590 [02:46<00:00,  3.54it/s]


Epoch 6: val_loss: 0.0641 val_precision: 0.9825 val_recall: 0.9900 val_f1: 0.9854 val_accuracy: 0.9826
Saving model..
Model saved.
